In [28]:
import sqlite3
con = sqlite3.connect("SQL_3.db")
cur = con.cursor()

In [29]:
cur.execute("DROP TABLE IF EXISTS Warehouses")
cur.execute("DROP TABLE IF EXISTS Boxes")

In [30]:
cur.execute("""
    CREATE TABLE Warehouses (
        Code INTEGER NOT NULL,
        Location VARCHAR(255) NOT NULL ,
        Capacity INTEGER NOT NULL,
        PRIMARY KEY (Code)
);
""")

cur.execute("""
    CREATE TABLE Boxes (
        Code CHAR(4) NOT NULL,
        Contents VARCHAR(255) NOT NULL ,
        Value REAL NOT NULL ,
        Warehouse INTEGER NOT NULL,
        PRIMARY KEY (Code)
);
""")

In [31]:
cur.executescript("""
 INSERT INTO Warehouses(Code,Location,Capacity) VALUES(1,'Chicago',3);
 INSERT INTO Warehouses(Code,Location,Capacity) VALUES(2,'Chicago',4);
 INSERT INTO Warehouses(Code,Location,Capacity) VALUES(3,'New York',7);
 INSERT INTO Warehouses(Code,Location,Capacity) VALUES(4,'Los Angeles',2);
 INSERT INTO Warehouses(Code,Location,Capacity) VALUES(5,'San Francisco',8);
 
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('0MN7','Rocks',180,3);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('4H8P','Rocks',250,1);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('4RT3','Scissors',190,4);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('7G3H','Rocks',200,1);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('8JN6','Papers',75,1);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('8Y6U','Papers',50,3);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('9J6F','Papers',175,2);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('LL08','Rocks',140,4);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('P0H6','Scissors',125,1);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('P2T6','Scissors',150,2);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('TU55','Papers',90,5);
""")

In [32]:
def run(cmd):
    r = cur.execute(cmd)
    display(r.fetchall())

## 3.1 Select all warehouses.

In [33]:
run("""
SELECT Location 
FROM Warehouses
""")

[('Chicago',),
 ('Chicago',),
 ('New York',),
 ('Los Angeles',),
 ('San Francisco',)]

## 3.2 Select all boxes with a value larger than $150.


In [34]:
run("""
SELECT Code, Contents 
FROM Boxes
WHERE Value > 150
""")

[('0MN7', 'Rocks'),
 ('4H8P', 'Rocks'),
 ('4RT3', 'Scissors'),
 ('7G3H', 'Rocks'),
 ('9J6F', 'Papers')]

## 3.3 Select all distinct contents in all the boxes.

In [35]:
run("""
SELECT DISTINCT(Contents) 
FROM Boxes
;
""")

[('Rocks',), ('Scissors',), ('Papers',)]

## 3.4 Select the average value of all the boxes.

In [36]:
run("""
SELECT ROUND(AVG(Value), 2) 
FROM Boxes
;
""")

[(147.73,)]

## 3.5 Select the warehouse code and the average value of the boxes in each warehouse.

In [37]:
run("""
SELECT Warehouse, ROUND(AVG(Value), 2) 
FROM Boxes
GROUP BY Warehouse
;
""")

[(1, 162.5), (2, 162.5), (3, 115.0), (4, 165.0), (5, 90.0)]

## 3.6 Same as previous exercise, but select only those warehouses where the average value of the boxes is greater than 150.


In [38]:
run("""
SELECT Warehouse, ROUND(AVG(Value), 2) 
FROM Boxes
WHERE Value > 150
GROUP BY Warehouse
;
""")

[(1, 225.0), (2, 175.0), (3, 180.0), (4, 190.0)]

## 3.7 Select the code of each box, along with the name of the city the box is located in.

In [39]:
run("""
SELECT B.Code, W.Location 
FROM Boxes B
JOIN Warehouses W
ON B.Warehouse = W.Code
;
""")

[('0MN7', 'New York'),
 ('4H8P', 'Chicago'),
 ('4RT3', 'Los Angeles'),
 ('7G3H', 'Chicago'),
 ('8JN6', 'Chicago'),
 ('8Y6U', 'New York'),
 ('9J6F', 'Chicago'),
 ('LL08', 'Los Angeles'),
 ('P0H6', 'Chicago'),
 ('P2T6', 'Chicago'),
 ('TU55', 'San Francisco')]

## 3.8 Select the warehouse codes, along with the number of boxes in each warehouse. 

In [40]:
run("""
SELECT Warehouse, COUNT(Warehouse) 
FROM Boxes 
GROUP BY Warehouse
;""")

[(1, 4), (2, 2), (3, 2), (4, 2), (5, 1)]

## 3.9 Select the codes of all warehouses that are saturated (a warehouse is saturated if the number of boxes in it is larger than the warehouse's capacity).


In [44]:
run("""
SELECT W.Code, W.Capacity, COUNT(B.Warehouse) AS number_of_boxes
FROM Boxes B
JOIN Warehouses W
ON B.Warehouse = W.Code
GROUP BY W.Code
HAVING W.Capacity < number_of_boxes
;""")

[(1, 3, 4)]

In [42]:
run("""
SELECT * 
FROM Boxes B
JOIN Warehouses W
ON B.Warehouse = W.Code
;
""")

[('0MN7', 'Rocks', 180.0, 3, 3, 'New York', 7),
 ('4H8P', 'Rocks', 250.0, 1, 1, 'Chicago', 3),
 ('4RT3', 'Scissors', 190.0, 4, 4, 'Los Angeles', 2),
 ('7G3H', 'Rocks', 200.0, 1, 1, 'Chicago', 3),
 ('8JN6', 'Papers', 75.0, 1, 1, 'Chicago', 3),
 ('8Y6U', 'Papers', 50.0, 3, 3, 'New York', 7),
 ('9J6F', 'Papers', 175.0, 2, 2, 'Chicago', 4),
 ('LL08', 'Rocks', 140.0, 4, 4, 'Los Angeles', 2),
 ('P0H6', 'Scissors', 125.0, 1, 1, 'Chicago', 3),
 ('P2T6', 'Scissors', 150.0, 2, 2, 'Chicago', 4),
 ('TU55', 'Papers', 90.0, 5, 5, 'San Francisco', 8)]

## 3.10 Select the codes of all the boxes located in Chicago.

In [43]:
run("""
SELECT B.Code, W.Code
FROM Boxes B
JOIN Warehouses W
ON B.Warehouse = W.Code
WHERE W.Location = 'Chicago'
;""")

[('4H8P', 1), ('7G3H', 1), ('8JN6', 1), ('9J6F', 2), ('P0H6', 1), ('P2T6', 2)]

[('9J6F',), ('P2T6',)]

## 3.11 Create a new warehouse in New York with a capacity for 3 boxes.

In [17]:
run("""
INSERT INTO Warehouses (Code, Location, Capacity)
VALUES (6, 'New York', 3)
;
""")

[]

## 3.12 Create a new box, with code "H5RT", containing "Papers" with a value of $200, and located in warehouse 2.

In [18]:
run("""
INSERT INTO Boxes (Code, Contents, Value, Warehouse)
VALUES ('H5RT', 'Papers', 200, 2)
;
""")

[]

## 3.13 Reduce the value of all boxes by 15%.


In [19]:
run("""
UPDATE Boxes 
SET Value = Value *.85 
;
""")

[]

## 3.14 Remove all boxes with a value lower than $100.


In [20]:
run("""
DELETE FROM Boxes 
WHERE Value < 100
;
""")

[]

## 3.15 Remove all boxes from saturated warehouses.

In [21]:
run("""
DELETE FROM Boxes
WHERE Warehouse IN
(SELECT Code
   	FROM Warehouses
   	WHERE Capacity <
   		(SELECT COUNT(*)
       	FROM Boxes
       	WHERE Warehouse = Warehouses.Code)
)
;""")

[]

In [22]:
run("""
SELECT * 
FROM Boxes B
JOIN Warehouses W
ON B.Warehouse = W.Code
;
""")

[('0MN7', 'Rocks', 153.0, 3, 3, 'New York', 7),
 ('4H8P', 'Rocks', 212.5, 1, 1, 'Chicago', 3),
 ('4RT3', 'Scissors', 161.5, 4, 4, 'Los Angeles', 2),
 ('7G3H', 'Rocks', 170.0, 1, 1, 'Chicago', 3),
 ('9J6F', 'Papers', 148.75, 2, 2, 'Chicago', 4),
 ('LL08', 'Rocks', 119.0, 4, 4, 'Los Angeles', 2),
 ('P0H6', 'Scissors', 106.25, 1, 1, 'Chicago', 3),
 ('P2T6', 'Scissors', 127.5, 2, 2, 'Chicago', 4),
 ('H5RT', 'Papers', 170.0, 2, 2, 'Chicago', 4)]

## 3.16 Add Index for column "Warehouse" in table "boxes"
    -- !!!NOTE!!!: index should NOT be used on small tables in practice

In [47]:
run("""
CREATE INDEX Index_Warehouse
ON Boxes (Warehouse)
;""")

[]

## 3.17 Print all the existing indexes
    -- !!!NOTE!!!: index should NOT be used on small tables in practice


In [48]:
run("""
SELECT * 
FROM SQLITE_MASTER 
WHERE type = "index";
""")

[('index', 'sqlite_autoindex_Boxes_1', 'Boxes', 2, None),
 ('index',
  'Index_Warehouse',
  'Boxes',
  5,
  'CREATE INDEX Index_Warehouse\nON Boxes (Warehouse)\n')]

## 3.18 Remove (drop) the index you added just
    -- !!!NOTE!!!: index should NOT be used on small tables in practice


In [49]:
run("""
DROP INDEX Index_Warehouse
;
""")

[]

In [50]:
con.close()

##

##